In [5]:
import random
import os
import re
import json
import numpy as np
import pandas as pd
import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from PIL import Image
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# from transformers import ViltProcessor, ViltModel
from transformers import CLIPProcessor, CLIPModel
# from transformers import ViltProcessor, ViltModel
from tqdm import trange
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
random_seed = 2020
random.seed(random_seed)
np.random.seed(random_seed)

2023-04-01 15:46:04.654078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 15:46:05.484548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-01 15:46:05.484629: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-01 15:46:05.484639: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [6]:
all_data = pd.read_csv('../data/combine_data_530.csv', low_memory=False, index_col=0)
all_data = all_data.fillna('0')
data_df = all_data
data_df

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,1,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,1,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1


In [ ]:
def get_img_data(img_file):
    if os.path.exists('../data/' + img_file) == True:
        return '../data/' + img_file
    else:
        return '../data/none_picture.jpg'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = CLIPProcessor.from_pretrained("../model/clip_base", truncation = True, padding = True)
model = CLIPModel.from_pretrained("../model/clip_base")
model.to(device)

In [ ]:
data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))
with torch.no_grad():
# img_mode_map = {'P': 0, 'L': 1, 'RGB': 2, 'CMYK': 3, 'RGBA': 4}
    inputs1 = processor(Image.open(data_df['img_file'][0]), data_df['Title'][0] + ' ' + data_df['Alltags'][0], return_tensors='pt', padding=True , truncation=True).to(device)
    outputs1 = model(**inputs1)
    clip_fea = outputs1.pooler_output
    for index in trange(486193):
        text = data_df['Title'][index + 1] + ', ' + data_df['Alltags'][index + 1]
        try:
            image = Image.open(data_df['img_file'][index + 1])  
            inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
            outputs = model(**inputs)
            vilt_fea1 = outputs.pooler_output
            vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
        except: 
            image = Image.open('../data/none_picture.jpg')
            inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
            outputs = model(**inputs)
            vilt_fea1 = outputs.pooler_output
            vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
vilt_fea_768 = pd.DataFrame(vilt_fea.cpu().detach().numpy())
vilt_fea_768.columns = ['viltvqa_fea_{}'.format(i) for i in range(768)]
vilt_fea.to_csv('../data/vilt_fea_vqa.csv', header=True, index=None)

In [7]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("../model/clip_base")
processor = CLIPProcessor.from_pretrained("../model/clip_base")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

In [8]:
probs

tensor([[0.9949, 0.0051]], grad_fn=<SoftmaxBackward0>)

In [13]:
outputs.text_embeds.size()

torch.Size([2, 512])

In [18]:
# outputs.text_embeds[0]

In [21]:
# outputs.image_embeds[0]

In [11]:
# outputs.pooler_output

AttributeError: 'CLIPOutput' object has no attribute 'pooler_output'

In [10]:
# outputs

CLIPOutput(loss=None, logits_per_image=tensor([[24.5701, 19.3049]], grad_fn=<TBackward0>), logits_per_text=tensor([[24.5701],
        [19.3049]], grad_fn=<MulBackward0>), text_embeds=tensor([[ 0.0148,  0.0070, -0.0234,  ..., -0.0508, -0.0438,  0.0033],
        [ 0.0087,  0.0258, -0.0387,  ..., -0.0547, -0.0242,  0.0112]],
       grad_fn=<DivBackward0>), image_embeds=tensor([[-9.7878e-03,  1.2770e-02, -2.7419e-02,  1.9676e-03, -5.9325e-03,
         -1.5613e-02, -1.2514e-02, -2.2669e-04,  4.3868e-02, -1.6322e-02,
          2.2630e-02, -3.5160e-02,  4.4749e-03, -1.2946e-02, -3.1524e-02,
         -1.1737e-02, -2.1543e-02, -2.7556e-02,  1.6562e-02,  4.5935e-03,
         -1.2106e-01, -3.0035e-03,  3.9024e-02, -3.0893e-02, -4.3866e-03,
          2.7598e-02,  2.2140e-02, -1.7065e-02,  1.4509e-02, -4.5196e-03,
         -7.1842e-03,  2.3971e-02, -6.8107e-03,  1.6382e-02, -5.3629e-02,
         -4.5554e-04,  2.5840e-02, -2.6581e-02,  1.7667e-02,  3.0216e-02,
         -9.3063e-03, -3.2082e-02,  6.6